In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import date, datetime, timedelta

In [ ]:
#
data = pd.read_csv("https://vnexpress.net/microservice/sheet/type/covid19_2021_by_day")
data = data[["day_full", "new_cases"]]

In [ ]:
#
data.fillna(0, inplace=True)
data["day_full"] = pd.to_datetime(data["day_full"], format="%Y/%m/%d")

In [ ]:
#
data = data.to_numpy()
series_covid19 = pd.Series(data=data[:, 1], index=data[:, 0])

In [ ]:
# fig = plt.figure(figsize=(18, 9))
# autocorrelation_plot(series_covid19)
# plt.show()

In [ ]:
#
series_covid19.index = series_covid19.index.to_period("D")
series_covid19 = series_covid19.astype(float)

In [ ]:
#
model = ARIMA(series_covid19, order=(25,1,0))
model_fit = model.fit()

# # summary of fit model
# print(model_fit.summary())
# # line plot of residuals
# residuals = pd.DataFrame(model_fit.resid)

# residuals.plot()
# plt.show()

In [ ]:
# # density plot of residuals
# residuals.plot(kind='kde')
# plt.show()

In [ ]:
# print(residuals.describe())

In [ ]:
# summary stats of residuals

In [ ]:
#
today = datetime.utcnow().date()
days = timedelta(days=1)
yesterday = today - days

train = series_covid19[:"2021-09-30"].values
test = series_covid19["2021-10-01":str(yesterday)].values
history = [x for x in train]
predictions = list()

In [ ]:
# walk-forward validation
for t in range(len(test)):
	model = ARIMA(history, order=(30,1,0), )
	model_fit = model.fit()
	output = model_fit.forecast()
	y_hat = output[0]
	predictions.append(y_hat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (y_hat, obs))

In [ ]:
#
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# # plot forecasts against actual outcomes
# plt.figure(figsize=(18, 9))
# plt.plot(test)
# plt.plot(predictions, color='red')
# plt.show()

In [ ]:
#
predictions = np.array(predictions)

In [ ]:
#
my_range = 60 # ADJUST RANGE HERE !!!
forecast = model_fit.forecast(steps=my_range)

In [ ]:
#
pred_fore = np.concatenate((predictions, forecast[1:]))

fromm = pd.to_datetime("2021-10-01", format="%Y-%m-%d")
to = yesterday + timedelta(days=my_range-1)
date = pd.Series(pd.date_range(fromm, to))

data =pd.DataFrame({"date": date, "forecast": pred_fore})

In [ ]:
#
data.to_csv("G:/My Drive/Covid19-in-VietNam/created_Covid19-in-VietNam_Forecast.csv", index=False)